# Timestamp Delta 및 관련 이슈

In [1]:
import pandas as pd
import pickle
import numpy as np

from tqdm import tqdm

In [2]:
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
} 

In [3]:
DATA_PATH = '/opt/ml/input/data/train_dataset/train_data.csv'
train_df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
train_df = train_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [4]:
DATA_PATH = '/opt/ml/input/data/train_dataset/test_data.csv'
test_df = pd.read_csv(DATA_PATH, dtype=dtype, parse_dates=['Timestamp'])
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

### Timestamp delta 계산 방법
- 기존 데이터의 Timestamp 는 해당 문제를 풀기 **시작한** 시점
- 동일 user, testId (시험지) 에서 Timestamp 기준으로 sorting 했을 때, 인접한 두 문제 중 뒤쪽 문제의 Timestamp 를 앞쪽 문제의 끝난 시점으로 사용해 그 차이를 초 단위로 계산


### Timestamp delta 가 0 으로 계산될 수 있는 경우
- Case 1: 한 testId 에 assessmentItemID 이 하나만 있을 때
- Case 2: 동일 user, 시험지에서 연속한 두 문항의 Timestamp 가 동일할 때

In [5]:
# Case 1
user_testid_groupby = train_df.groupby(['testId', 'userID']).agg({
    'assessmentItemID': 'count'
})
user_testid_groupby[user_testid_groupby['assessmentItemID'] == 1]
# 없는 것 같다..!

,,assessmentItemID
testId,userID,


In [6]:
# Case 1 - test data 는?
user_testid_groupby = test_df.groupby(['testId', 'userID']).agg({
    'assessmentItemID': 'count'
})
user_testid_groupby[user_testid_groupby['assessmentItemID'] == 1]
# 얘도 없는 것 같다..!

,,assessmentItemID
testId,userID,


In [7]:
# Case 2 가 생각보다 많다. 예시로 아래에 row 122, 123 의 Timestamp 가 동일하다
# 여러 문제의 풀기 시작한 시점이 같다는 것은 한 문제 아래에 딸린 문제라는 의미인지?
train_df[120:126]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
120,0,A080016005,A080000016,1,2020-05-07 00:51:29,4669
121,0,A080016006,A080000016,1,2020-05-07 00:51:48,4664
122,0,A080016007,A080000016,1,2020-05-07 00:52:47,4672
123,0,A080016008,A080000016,0,2020-05-07 00:52:47,4672
124,0,A060024001,A060000024,1,2020-05-07 01:20:10,595
125,0,A060024002,A060000024,1,2020-05-07 01:20:16,594


### Timestamp delta 가 매우 크게 계산되는 경우
- Case 1: 동일한 시험지를 꼭 하루에 연속해서 풀지 않을 때
- Case 2: 동일한 시험지를 여러 번 풀 때

Threshold 를 넘어가면 값을 초기화하거나 (Case 2) 무시하고 다른 통계량으로 대체 (Case 1) 하기?

In [8]:
# 동일한 시험지를 꼭 하루에 연속해서 풀지 않을 수도 있다.
# 아래 row 15193 에서 15194 로 넘어갈 때 하루 넘게 걸렸다
train_df[15193:15196]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
15193,23,A080005002,A080000005,1,2020-01-07 10:14:34,4657
15194,23,A080005001,A080000005,0,2020-01-08 10:48:38,4657
15195,23,A080005003,A080000005,1,2020-01-08 10:49:06,4657


In [9]:
# 아래의 경우는 동일 시험지를 두달 후 다시 풀고 있음..!
# 지금은 동일 user, testId 라면 인접한 Timestamp 의 delta 를 무조건 계산하므로,
# row 2078016 에 매우 큰 값이 부여된다
train_df[2078007:2078020]   

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
2078007,4631,A040027001,A040000027,1,2020-01-16 08:49:34,2053
2078008,4631,A040027002,A040000027,1,2020-01-16 08:49:46,2053
2078009,4631,A040027003,A040000027,1,2020-01-16 08:49:54,2053
2078010,4631,A040027004,A040000027,1,2020-01-16 08:50:12,2053
2078011,4631,A040027005,A040000027,0,2020-01-16 08:50:33,2053
2078012,4631,A040027006,A040000027,0,2020-01-16 08:52:27,2053
2078013,4631,A040027007,A040000027,1,2020-01-16 08:53:14,11214
2078014,4631,A040027008,A040000027,0,2020-01-16 08:54:51,11214
2078015,4631,A040027009,A040000027,0,2020-01-16 08:54:57,11214
2078016,4631,A040027010,A040000027,0,2020-01-16 08:55:04,11214
